# Assignment

In this assignment we will train a convolutional neural network (CNN) on the CIFAR-10 dataset.

This assignment is part of the class **Introduction to Deep Learning for Medical Imaging** at University of California Irvine (CS190); more information can be found: https://github.com/peterchang77/dl_tutor/tree/master/cs190.

### Submission

Once complete, the following items must be submitted:

* final `*.ipynb` notebook (push to https://github.com/[username]/cs190/cnn/assignment.ipynb)
* final trained `*.hdf5` model file
* final compiled `*.csv` file with performance statistics

# Google Colab

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

# Environment

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [1]:
# --- Install jarvis (only in Google Colab or local runtime)
% pip install jarvis-md

     |████████████████████████████████| 81kB 5.3MB/s 
     |████████████████████████████████| 645kB 10.8MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### Imports

Use the following lines to import any additional needed libraries:

In [2]:
import os, numpy as np, pandas as pd
from tensorflow import losses, optimizers
from tensorflow.keras import Input, Model, models, layers
from jarvis.train import datasets
from tensorflow.keras import regularizers

# Data

As in the tutorial, data for this assignment will consist of the CIFAR-10 dataset comprising 10 different everyday objects (airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck). The following lines of code will:

1. Download the dataset (if not already present) 
2. Prepare the necessary Python generators to iterate through dataset
3. Prepare the corresponding Tensorflow Input(...) objects for model definition

In [3]:
# --- Download dataset
datasets.download(name='cifar')

# --- Prepare generators and model inputs
gen_train, gen_valid, client = datasets.prepare(name='cifar')
inputs = client.get_inputs(Input)

[ 2021-04-17 22:48:12 ] [====================] 100.000% : Iterating | 000001    

# Training

In this assignment we will train a basic convolutional neural network on the CIFAR-10 dataset. At minumum you must include the following baseline techniques covered in the tutorial:

* convolutional operations
* batch normalization
* activation function
* subsampling

You are also **encouraged** to try different permuations and customizations to achieve optimal validation accuracy.

### Define the model

In [4]:
inputs = {}
inputs['dat'] = Input(shape=(32,32,3))

kwargs = {
    'kernel_size': (3, 3),
    'padding': 'same', 
    'kernel_regularizer' : regularizers.l2(0.01)}

conv = lambda x, filters, strides : layers.Conv2D(filters=filters, strides=strides, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.ReLU()(x)

pool2 = lambda x : layers.MaxPooling2D(pool_size=(2, 2), strides = (2, 2))(x)
# pool2 = lambda x : layers.MaxPooling2D(pool_size=(2, 2))(x)

conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=(2, 2))))

drop = layers.Dropout(rate=0.25)

# l1 = conv1(16, inputs['dat'])
# l2 = conv1(16, l1)
# l3 = pool2(l2)
# l4 = drop(l3)
# l5 = conv1(32, l4)
# l6 = conv1(32, l5)
# l7 = conv2(32, l6)
# l8 = drop(l7)
# l9 = conv1(64, l8)
# l10 = conv1(64, l9)
# l11 = pool2(l10)

l1 = conv1(32, inputs['dat'])
l2 = conv1(32, l1)
l3 = pool2(l2)
l4 = drop(l3)
l5 = conv1(64, l4)
l6 = conv1(64, l5)
l7 = pool2(l6)

f0 = layers.Flatten()(l7)
# f0 = layers.Conv2D(filters=48, kernel_size = (4,4), strides=(1,1), padding='valid')(l4)
f1 = drop(f0)
f2 = layers.Dense(128, activation='relu')(f1)

logits = {}
logits['class'] = layers.Dense(10, name='class')(f2)

# --- Create model
model = Model(inputs=inputs, outputs=logits)

### Compile the model

In [5]:
# --- Compile model
model.compile(
    optimizer=optimizers.Adam(learning_rate=2e-4), 
    loss={'class': losses.SparseCategoricalCrossentropy(from_logits=True)}, 
    metrics={'class': 'sparse_categorical_accuracy'})

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 32)   896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 32)   128         conv2d[0][0]                     
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 32, 32, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

### Train the model

In [6]:
model.fit(
    x=gen_train, 
    steps_per_epoch=750, 
    epochs=20,
    validation_data=gen_valid,
    validation_steps=750,
    validation_freq=4)

Epoch 1/20
750/750 [==============================] - 56s 30ms/step - loss: 3.3022 - sparse_categorical_accuracy: 0.3186
Epoch 2/20
750/750 [==============================] - 23s 30ms/step - loss: 2.4014 - sparse_categorical_accuracy: 0.4898
Epoch 3/20
750/750 [==============================] - 23s 30ms/step - loss: 1.9041 - sparse_categorical_accuracy: 0.5702
Epoch 4/20
750/750 [==============================] - 42s 56ms/step - loss: 1.6225 - sparse_categorical_accuracy: 0.6066 - val_loss: 1.4527 - val_sparse_categorical_accuracy: 0.6398
Epoch 5/20
750/750 [==============================] - 23s 30ms/step - loss: 1.4016 - sparse_categorical_accuracy: 0.6483
Epoch 6/20
750/750 [==============================] - 23s 31ms/step - loss: 1.3014 - sparse_categorical_accuracy: 0.6577
Epoch 7/20
750/750 [==============================] - 23s 31ms/step - loss: 1.1866 - sparse_categorical_accuracy: 0.6788
Epoch 8/20
750/750 [==============================] - 41s 55ms/step - loss: 1.1387 - sparse_

In [7]:
model.save('./cnn.hdf5')

# del model
# model = models.load_model('./cnn.hdf5', compile=False)

In [8]:
model = models.load_model('./cnn.hdf5', compile=False)

# Evaluation

Based on the tutorial discussion, use the following cells to check your algorithm performance. Consider loading a saved model and running prediction using `model.predict(...)` on the data aggregated via a test generator.

In [9]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True)

# --- Aggregate all examples
xs = []
ys = []

# use test generator so you don't run infintely 
for x, y in test_valid:
    xs.append(x['dat'])
    ys.append(y['class'])

xs = np.concatenate(xs)
ys = np.concatenate(ys)

# --- Predict
logits = model.predict(xs)

if type(logits) is dict:
    logits = logits['class']

# --- Argmax
pred = np.argmax(logits, axis=1)

[ 2021-04-17 23:02:56 ] [====================] 100.000% : Iterating | 012000    

**Note**: this cell is used only to check for model performance. It will not be graded. Once you are satisfied with your model, proceed to submission of your assignment below.

### Results

When ready, create a `*.csv` file with your compiled **validation** cohort statistics. There is no need to submit training performance accuracy. As in the tutorial, ensure that there are at least three columns in the `*.csv` file:

* true (ground-truth)
* pred (prediction)
* corr (correction prediction, True or False)

In [10]:
# --- Create *.csv
df = pd.DataFrame(index=np.arange(pred.size))

# --- Define columns
df['true'] = ys[:, 0]
df['pred'] = pred
df['corr'] = df['true'] == df['pred']

# --- Print accuracy
print(df['corr'].mean())

0.73975


In [11]:

MOUNT_ROOT = './gdrive/MyDrive'
                              
# --- Serialize *.csv
fname = '{}/models/cnn/results.csv'.format(MOUNT_ROOT)
os.makedirs(os.path.dirname(fname), exist_ok=True)
df.to_csv(fname)

# Submission

Use the following line to save your model for submission:

In [ ]:
# --- Serialize a model
model.save('./cnn.hdf5')

### Canvas

Once you have completed this assignment, download the necessary files from Google Colab and your Google Drive. You will then need to submit the following items:

* final (completed) notebook: `[UCInetID]_assignment.ipynb`
* final (results) spreadsheet: `[UCInetID]_results.csv`
* final (trained) model: `[UCInetID]_model.hdf5`

**Important**: please submit all your files prefixed with your UCInetID as listed above. Your UCInetID is the part of your UCI email address that comes before `@uci.edu`. For example, Peter Anteater has an email address of panteater@uci.edu, so his notebooke file would be submitted under the name `panteater_notebook.ipynb`, his spreadshhet would be submitted under the name `panteater_results.csv` and and his model file would be submitted under the name `panteater_model.hdf5`.